# Preliminary Setup

In [1]:
!pip install wandb -q

     |████████████████████████████████| 1.9 MB 30.8 MB/s 
     |████████████████████████████████| 168 kB 70.7 MB/s 
     |████████████████████████████████| 182 kB 59.5 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 168 kB 70.6 MB/s 
     |████████████████████████████████| 166 kB 65.9 MB/s 
     |████████████████████████████████| 166 kB 83.4 MB/s 
     |████████████████████████████████| 162 kB 83.3 MB/s 
     |████████████████████████████████| 162 kB 57.8 MB/s 
     |████████████████████████████████| 158 kB 97.9 MB/s 
     |████████████████████████████████| 157 kB 75.6 MB/s 
     |████████████████████████████████| 157 kB 75.3 MB/s 
     |████████████████████████████████| 157 kB 82.8 MB/s 
     |████████████████████████████████| 157 kB 76.6 MB/s 
     |████████████████████████████████| 157 kB 81.7 MB/s 
     |████████████████████████████████| 157 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 76.0 MB/s 
     |██████████

In [1]:
import wandb

In [2]:
wandb.login(key="d90e12ac420e91892a07ed5f8e37b39f5112f1a8")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cdgr (dl-studygroup). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
!pip install python-Levenshtein

!pip install torchsummaryX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 34.0 MB/s 
     |████████████████████████████████| 2.2 MB 50.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 14.49 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14102, done.        
remote: Counting objects: 100% (415/415), done.        
remote: Compressing objects: 100% (290/290), done.        
remote: Total 14102 (delta 12

In [3]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
#import ctcdecode
import Levenshtein
#from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [4]:
# import os.path as path 
# if not path.exists("/content/gdrive"):
#   !sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#   !sudo apt-get update -qq 2>&1 > /dev/null
#   !sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
#   !google-drive-ocamlfuse

#   !sudo apt-get install -qq w3m
#   !xdg-settings set default-web-browser w3m.desktop
#   %cd /content
#   !mkdir gdrive
#   %cd gdrive
#   !mkdir MyDrive
#   %cd ..
#   %cd ..
#   !google-drive-ocamlfuse /content/gdrive/MyDrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Loading and preprocessing

In [8]:
!unzip -q /content/drive/MyDrive/f0176.zip
!ls

ctcdecode  drive  f0176  f0176.zip  sample_data


In [4]:
CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict
mapping = CMUdict_ARPAbet
LABELS = ARPAbet

In [5]:
print(PHONEMES)
print(LABELS)

['', '[SIL]', 'NG', 'F', 'M', 'AE', 'R', 'UW', 'N', 'IY', 'AW', 'V', 'UH', 'OW', 'AA', 'ER', 'HH', 'Z', 'K', 'CH', 'W', 'EY', 'ZH', 'T', 'EH', 'Y', 'AH', 'B', 'P', 'TH', 'DH', 'AO', 'G', 'L', 'JH', 'OY', 'SH', 'D', 'AY', 'S', 'IH', '[SOS]', '[EOS]']
[' ', '-', 'G', 'f', 'm', '@', 'r', 'u', 'n', 'i', 'W', 'v', 'U', 'o', 'a', 'R', 'h', 'z', 'k', 'C', 'w', 'e', 'Z', 't', 'E', 'y', 'A', 'b', 'p', 'T', 'D', 'c', 'g', 'l', 'j', 'O', 'S', 'd', 'Y', 's', 'I', '[SOS]', '[EOS]']


In [6]:
phoneme_index_mapping = dict()
for i, phoneme in enumerate(PHONEMES):
  phoneme_index_mapping[phoneme] = i
print(phoneme_index_mapping)

{'': 0, '[SIL]': 1, 'NG': 2, 'F': 3, 'M': 4, 'AE': 5, 'R': 6, 'UW': 7, 'N': 8, 'IY': 9, 'AW': 10, 'V': 11, 'UH': 12, 'OW': 13, 'AA': 14, 'ER': 15, 'HH': 16, 'Z': 17, 'K': 18, 'CH': 19, 'W': 20, 'EY': 21, 'ZH': 22, 'T': 23, 'EH': 24, 'Y': 25, 'AH': 26, 'B': 27, 'P': 28, 'TH': 29, 'DH': 30, 'AO': 31, 'G': 32, 'L': 33, 'JH': 34, 'OY': 35, 'SH': 36, 'D': 37, 'AY': 38, 'S': 39, 'IH': 40, '[SOS]': 41, '[EOS]': 42}


In [7]:
class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root_dir, subset="train"): 
        '''
        Initializes the dataset.
        '''
        self.root_dir = root_dir
        self.subset = subset
        mfcc_dir = f"{root_dir}/{subset}.npy"
        trans_dir = f"{root_dir}/{subset}_labels.npy"

        self.mfccs = torch.from_numpy(np.load(mfcc_dir))
        self.transcripts = torch.from_numpy(np.load(trans_dir))

        self.size = len(self.mfccs)

    def __len__(self):
        return self.size

    def __getitem__(self, index):
        return self.mfccs[index], self.transcripts[index]

    def collate_fn(self, batch):
        mfccs = [u[0] for u in batch]
        transcripts = [u[1] for u in batch]

        mfccLens = torch.tensor([len(m) for m in mfccs])
        transcriptLens = torch.tensor([len(t) for t in transcripts])

        mfccs = pad_sequence(
          mfccs, batch_first=True
        )
        transcripts = pad_sequence(
          transcripts, batch_first=True
        )
        return mfccs, transcripts, mfccLens, transcriptLens
       

In [8]:
BATCH_SIZE = 64

root = 'f0176/half2' 

In [9]:
import gc 
gc.collect()

0

In [10]:
train_data = AudioDataset(root, subset="train")
val_data = AudioDataset(root, subset="dev")

train_loader = torch.utils.data.DataLoader(train_data, num_workers = 4,
                                           batch_size = BATCH_SIZE, pin_memory = True,
                                           shuffle = True, collate_fn = train_data.collate_fn)

val_loader = torch.utils.data.DataLoader(val_data, num_workers = 2,
                                         batch_size = BATCH_SIZE, pin_memory = True,
                                         shuffle = False, collate_fn = val_data.collate_fn)

print("Batch size: ", BATCH_SIZE)
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))

Batch size:  64
Train dataset samples = 8000, batches = 125
Val dataset samples = 800, batches = 13


In [11]:
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    print(y[0])
    break 

torch.Size([64, 176, 26]) torch.Size([64, 23]) torch.Size([64]) torch.Size([64])
tensor([41,  1, 38,  4, 32, 24, 23, 40,  2, 39, 26,  4, 20, 26, 23, 40,  8, 11,
        14, 33, 11, 37, 42])


# Model Definition

In [12]:
OUT_SIZE = len(LABELS)
OUT_SIZE

43

In [13]:
class LSTMLayers(nn.Module):
    """
        (Bidirectional) LSTM Module
    """
    def __init__(
            self,
            dim_in: int=512,
            hidden_dims: list=[256, 256],
            num_layers: list=[4, 4],
            bidirectionals: list=[True, True],
            dropouts: list=[0.3, 0.3]
        ):
        """
            Note: locked dropout only applied once to concat layers at the end
                    to apply one locked dropout per lstm, use LockedGroupedLSTM instead
        """
        super().__init__()
        self.dims = [dim_in] + hidden_dims
        self.num_layers = num_layers
        self.bidirectionals = bidirectionals
        self.dropouts = dropouts
        assert (
            len(self.dims) 
            == len(self.num_layers) + 1 
            == len(self.bidirectionals) + 1
        )

        self.streamline = nn.ModuleList()
        for l, dim in enumerate(self.dims[:-1]):
            input_dim = (
                dim * 2 if l > 0 and self.bidirectionals[l]
                else dim
            )
            lstm = nn.LSTM(
                input_size=input_dim,
                hidden_size=self.dims[l + 1],
                num_layers=self.num_layers[l],
                bidirectional=self.bidirectionals[l],
                dropout=self.dropouts[l],
                batch_first=True
            )
            self.streamline.append(lstm)

        
    def forward(self, x, lx):
        xx = pack_padded_sequence(
            x, lengths=lx, batch_first=True, enforce_sorted=False
        )
        for layer in self.streamline:
            xx, _ = layer(xx)
        xx, lx = pad_packed_sequence(
            xx, batch_first=True
        )
        return xx, lx

In [14]:
class ClsHead(nn.Module):
    """
        Final Linear & Classification Module
    """
    def __init__(
            self, 
            dim_in: int=256,
            dims: list=[512],
            num_labels: int=43,
            dropout: float=0.2
        ):
        super().__init__()
        # bookkeeping
        self.dims = [dim_in] + dims + [num_labels]
        self.linears = nn.ModuleList()
        # linear(s)
        for i, dim in enumerate(self.dims[:-1]):
            self.linears.append(
                nn.Linear(
                    in_features=dim,
                    out_features=self.dims[i + 1]
                )
            )
            if i < len(self.dims) - 2:
                self.linears.append(nn.Dropout(dropout))
                self.linears.append(nn.GELU())
        assert len(self.linears) == 3 * len(self.dims) - 5
        # logsoftmax
        self.softmax = nn.Softmax(dim=-1)


    def forward(self, x):
        for layer in self.linears:
            x = layer(x)
        x = self.softmax(x)
        return x

In [15]:
class Baseline(nn.Module):
    """
        Partial network from BiLSTM to Classification.
    """
    def __init__(
            self,
            input_dim: int,
            lstm_cfgs: dict,
            cls_cfgs: dict
        ):
        super().__init__()
        self.configs = {
            'mfcc_dim': input_dim,
            'lstm': lstm_cfgs,
            'cls': cls_cfgs
        }

        # dim_in for lstm is the original MFCC dimensions
        self.lstm = LSTMLayers(dim_in=input_dim, **lstm_cfgs)

        # dim_in for cls is the final hidden dimension of LSTMs
        cls_dim_in = lstm_cfgs['hidden_dims'][-1]

        # alter the dimension based on whether bidirectional is enabled
        cls_dim_in *= 2 if lstm_cfgs['bidirectionals'][-1] else 1
        self.cls = ClsHead(dim_in=cls_dim_in, **cls_cfgs)
    

    def forward(self, x, lx):
        # (bi)lstm layers
        xx, lx = self.lstm(x, lx)
        # cls layers
        xx = self.cls(xx)
        return xx, lx

In [31]:
torch.manual_seed(0)
MFCC_DIM = 26
HIDDEN_DIMS = [128, 128]
NUM_LAYERS = [2, 2]
BIDIRECTIONALS = [True, True]
LINEAR_DIMS = [256]
DROPOUTS = [0.2, 0.2]
NUM_PHONEMES = 43


LSTM_CFGS = {
    'hidden_dims': HIDDEN_DIMS,
    'num_layers': NUM_LAYERS,
    'bidirectionals': BIDIRECTIONALS,
    'dropouts': DROPOUTS
}

CLS_CFGS = {
    'dims': LINEAR_DIMS,
    'num_labels': NUM_PHONEMES
}

model = Baseline(
    input_dim=MFCC_DIM,
    lstm_cfgs=LSTM_CFGS,
    cls_cfgs=CLS_CFGS
)
model = model.to(device)
print(f"\n\nModel Summary for [LSTMLayers]:\n{summary(model, x.to(device), lx)}\n\n")

                         Kernel Shape    Output Shape    Params Mult-Adds
Layer                                                                    
0_lstm.streamline.LSTM_0            -    [11264, 256]  555.008k  550.912k
1_lstm.streamline.LSTM_1            -    [11264, 256]  790.528k  786.432k
2_cls.linears.Linear_0     [256, 256]  [64, 176, 256]   65.792k   65.536k
3_cls.linears.Dropout_1             -  [64, 176, 256]         -         -
4_cls.linears.GELU_2                -  [64, 176, 256]         -         -
5_cls.linears.Linear_3      [256, 43]   [64, 176, 43]   11.051k   11.008k
6_cls.Softmax_softmax               -   [64, 176, 43]         -         -
--------------------------------------------------------------------------
                         Totals
Total params          1.422379M
Trainable params      1.422379M
Non-trainable params        0.0
Mult-Adds             1.413888M


Model Summary for [LSTMLayers]:
                         Kernel Shape    Output Shape    Params  

# Baseline Model Training

## Loss functions, Optimizer, Decoder Setup

In [17]:
baseline_config = {
    "beam_width" : 3,
    "lr" : 2e-3,
    "epochs" : 50,
    "scheduler": "ReduceLR",
    "layers": "LSTM(128, 128) with dropout, Linear(256)",
    }

In [ ]:
run = wandb.init(
    reinit = True,
    project = "project",
    config = baseline_config
)

levenshtein distance,█▅▅▅▅▃▁
train loss,█▄▃▃▃▂▁
validation loss,███▇▅▃▁
levenshtein distance,18.09255
train loss,2.6327
validation loss,2.55744


In [18]:
optimizer =  torch.optim.AdamW(model.parameters(), lr=baseline_config['lr'], weight_decay=1e-5) # What goes in here?

In [ ]:
from torch.nn.modules.loss import CTCLoss

criterion = CTCLoss()


decoder = CTCBeamDecoder(
    LABELS,
    beam_width = config["beam_width"],
    num_processes = 4,
    log_probs_input = True
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 2, factor = 0.5, min_lr = 2e-6, mode = "min")

scaler = torch.cuda.amp.GradScaler()

In [ ]:
def calculate_levenshtein(h, y, lh, ly, decoder, labels):

    beam_results, beam_scores, timesteps, out_lens = decoder.decode(h, seq_lens = lh)

    batch_size = out_lens.shape[0]
    distance = 0

    for i in range(batch_size):
      h_sliced = beam_results[i][0][:out_lens[i][0]]
      y_sliced = y[i][:ly[i]]
      h_string = "".join([labels[n] for n in h_sliced])
      y_string = "".join([labels[n] for n in y_sliced])
      distance += Levenshtein.distance(h_string, y_string)

    distance /= batch_size

    return distance

In [ ]:
# SANITY CHECK FOR BASELINE MODEL

with torch.no_grad():
  for i, data in enumerate(train_loader):

      mfccs, phonemes, len_mfccs, len_phonemes = data
      mfccs, phonemes = mfccs.to(device), phonemes.to(device) 

      with torch.cuda.amp.autocast():
        out, lens = model(mfccs, len_mfccs)

      print("Output of model: ", out.shape)

      loss = criterion(torch.log(out).permute(1,0,2), phonemes, lens, len_phonemes) # What goes in here?
      print(f"loss: {loss}")

      distance = calculate_levenshtein(torch.log(out), phonemes, lens, len_phonemes, decoder, LABELS, debug = False)
      print(f"lev-distance: {distance}")

      break

Output of model:  torch.Size([64, 44, 43])
loss: 5.496857166290283
lev-distance: 29.296875


## Training and Evaluation

In [ ]:
torch.cuda.empty_cache()
def evaluate(data_loader, model, epoch):
    model.eval()
    dist = 0
    loss = 0
    batch_bar = tqdm(total=len(data_loader), dynamic_ncols=True, leave=False, position=0, desc='Val') 

    phone_true_list = []
    phone_pred_list = []

    for i, data in enumerate(data_loader):

        mfccs, phonemes, len_mfccs, len_phonemes = data
        mfccs, phonemes = mfccs.to(device), phonemes.to(device) 

        with torch.inference_mode():
          out, lens = model(mfccs, len_mfccs)
    

        loss = criterion(torch.log(out).permute(1,0,2), phonemes, lens, len_phonemes)

        dist += calculate_levenshtein(torch.log(out), phonemes, lens, len_phonemes, decoder, LABELS, debug = False)

        del mfccs, phonemes, out
        torch.cuda.empty_cache()
        
    dist /= len(data_loader)

    return loss, dist


In [ ]:
def train_step(train_loader, model, optimizer, criterion, scheduler, scaler):
  
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    train_loss = 0

    for i, data in enumerate(train_loader):
        mfccs, phonemes, len_mfccs, len_phonemes = data
        mfccs, phonemes = mfccs.to(device), phonemes.to(device) 

        with torch.cuda.amp.autocast():
          out, lens = model(mfccs, len_mfccs)

        loss = criterion(torch.log(out).permute(1,0,2), phonemes, lens, len_phonemes)

        batch_bar.set_postfix(
            loss = f"{train_loss/ (i+1):.4f}",
            lr = f"{optimizer.param_groups[0]['lr']}"
        )

        train_loss += loss
        batch_bar.update()

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    
    batch_bar.close()
    train_loss /= len(train_loader)

    return train_loss

In [ ]:
torch.cuda.empty_cache()
gc.collect()

best_val_dist = float('inf')

name = "baseline"

train_losses = []
validation_losses = []

for epoch in range(baseline_config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, baseline_config['epochs']))
    train_loss = train_step(train_loader, model, optimizer, criterion, scheduler, scaler)
    print("\tTrain Loss: {:.4f}".format(train_loss))
    
    validation_loss, levenshtein_distance = evaluate(val_loader, model, epoch)
    print("\tLevenshtein Distance: {}".format(levenshtein_distance))

    validation_losses.append(validation_loss)

    scheduler.step(levenshtein_distance)

    if levenshtein_distance < best_val_dist:
      path = "/content/drive/MyDrive/checkpoint_{}.pth".format(name) 
      print("Saving model")
      torch.save({'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'val_dist': levenshtein_distance, 
                  'epoch': epoch}, path)
      
      best_val_dist = levenshtein_distance
      wandb.save('checkpoint.pth')
    
    wandb.log({"train loss": train_loss, "levenshtein distance": levenshtein_distance, "validation loss": validation_loss})

run.finish()

In [19]:
path = "/content/drive/MyDrive/{}.pth".format("checkpoint_baseline_final")
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

best_val_dist = checkpoint['val_dist']

In [20]:
print("Best Validation Distance for Baseline Model: ", best_val_dist)

Best Validation Distance for Baseline Model:  6.688701923076923


In [32]:
path = "/content/drive/MyDrive/checkpoint_first.pth"
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

best_val_dist = checkpoint['val_dist']

In [33]:
print("Best Validation Distance for Baseline Model: ", best_val_dist)

Best Validation Distance for Baseline Model:  7.1983173076923075


# RL Model Training

## Loss function, optimizer setup

In [21]:
finetuning_config = {
    "lr" : 1e-4,
    "epochs" : 50,
    "scheduler": "ReduceLR",
    "layers": "LSTM(128, 128) with dropout, Linear(256)",
    "num_samples": 2
    }

In [139]:
run.finish()

In [37]:
run = wandb.init(
    reinit = True,
    project = "11785-project",
    config = finetuning_config
)

In [23]:
class MultinomialDecode:
    def __init__(self, num_samples=1):
        self.num_samples = num_samples
    
    def decode(self, logits, debug=False):
      samples = torch.multinomial(logits, self.num_samples, replacement=True)
      if debug:
        print(logits[0], samples[0])
      predicted = []
      for i in range(self.num_samples):
        pred = ""
        prev = ""
        for t in range(samples.shape[0]):
          curr = mapping[PHONEMES[samples[t, i].item()]]

          if t == 0:
              pred += curr
              prev = curr
          else:
              if curr != prev and curr !=" ":
                  pred += curr
                  prev = curr
              if curr == " ":
                  prev = ""
        predicted.append(pred)
      samples =  torch.arange(logits.size(0)).to(device)[:, None] * logits.size(1) + samples.to(device)
      # print(samples)
      # print(logits)
      # print(torch.take(logits, samples))
      # print(torch.grad_fn)
      return predicted, torch.take(logits, samples)

In [24]:
class RLLoss:
    def __init__(self, decoder):
        self.decoder = decoder

    def __call__(self, logits, target, input_lengths, target_lengths, return_dist=False):
      return self.forward(logits, target, input_lengths, target_lengths, return_dist)

    def forward(self, logits, target, input_lengths, target_lengths, return_dist=False):
      batch_size, timesteps, _ = logits.shape

      total_loss = 0.0

      total_dist = 0.0

      debug = False
      for i in range(batch_size):
        probs = logits[i, :input_lengths[i], :]
        if i == 1:
          debug = False
        else:
          debug = False
        predicted, probs = self.decoder.decode(probs, debug)


        actual = "".join([mapping[PHONEMES[j]] for j in target[i, :target_lengths[i]]])

        sample_distances = []


        for sample in range(len(predicted)):
          sample_distances.append(Levenshtein.distance(predicted[sample], actual))
          # print('Predicted : ', predicted[sample])
          # print('Actual: ', actual)
          # print()
          #print(sample_distances[-1])
        
        total_dist += sum(sample_distances) / len(predicted)
        sample_distances = torch.tensor(sample_distances, dtype=torch.float, requires_grad=True).to(device)
        sample_distances = sample_distances - torch.mean(sample_distances)
        total_loss += torch.sum(sample_distances * torch.sum(torch.log(probs), axis=0)) / len(predicted)

      total_loss = total_loss / batch_size
      total_dist = total_dist / batch_size
      if return_dist:
        return total_loss, total_dist

      return total_loss

In [35]:
decoder = MultinomialDecode(finetuning_config["num_samples"])

criterion = RLLoss(decoder)

optimizer =  torch.optim.AdamW(model.parameters(), lr=finetuning_config['lr'], weight_decay=1e-4) # What goes in here?

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 5, factor = 0.2, min_lr = 1e-7, mode = "min", verbose=True)

scaler = torch.cuda.amp.GradScaler()

In [129]:
# SANITY CHECK FOR FINE-TUNING
model = model.to(device)

with torch.no_grad():
  for i, data in enumerate(train_loader):
      
      mfccs, phonemes, len_mfccs, len_phonemes = data
      mfccs, phonemes = mfccs.to(device), phonemes.to(device) 
      print(mfccs.shape)
      print(phonemes.shape)

      with torch.cuda.amp.autocast():
        out, lens = model(mfccs, len_mfccs)

      print("Output of model: ", out.shape)

      loss, dist = criterion(out, phonemes, lens, len_phonemes, return_dist=True)
      print(f"loss: {loss}")
      print(f"lev dist: {dist}")

      break

torch.Size([64, 176, 26])
torch.Size([64, 23])
Output of model:  torch.Size([64, 176, 43])
Predicted :  [SOS]-AtlazAvstosSi@tAdInRmn[EOS]
Actual:  [SOS]-ItwazAsosietAdInDErm[EOS]

Predicted :  [SOS]-ItvwIzAvsAtSi@tAdInRm[EOS]
Actual:  [SOS]-ItwazAsosietAdInDErm[EOS]

Predicted :  [SOS]-DATmGizUr-kEptInDAos[EOS]
Actual:  [SOS]-nATIGiziRk-k@ptAnEmo[EOS]

Predicted :  [SOS]-DATAmizyIrkEptInDEmo[EOS]
Actual:  [SOS]-nATIGiziRk-k@ptAnEmo[EOS]

Predicted :  [SOS]-mIsIzr@pejpYkyApYzAkra[EOS]
Actual:  [SOS]-mIsIzr@mpejakyApYzAp[EOS]

Predicted :  [SOS]-mIs@zr@mdpedpYkyApYzAvpa[EOS]
Actual:  [SOS]-mIsIzr@mpejakyApYzAp[EOS]

Predicted :  [SOS]-DAtmYonlitritItneWzhn[EOS]
Actual:  [SOS]-AndYonlitritItnW@zIt[EOS]

Predicted :  [SOS]-D@mYAntlitritItnWAzA[EOS]
Actual:  [SOS]-AndYonlitritItnW@zIt[EOS]

Predicted :  [SOS]-AndhitukmYhEdInhIzl[EOS]
Actual:  [SOS]-AndhitUkmYh@ndInhIzl[EOS]

Predicted :  [SOS]-AndhitukmYhendInhIzla[EOS]
Actual:  [SOS]-AndhitUkmYh@ndInhIzl[EOS]

Predicted :  [SOS]-wwRznhhTo-

## Training and Evaluation

In [26]:
torch.cuda.empty_cache()
def evaluate(data_loader, model, epoch, decoder):
    model.eval()
    dist = 0
    loss = 0
    batch_bar = tqdm(total=len(data_loader), dynamic_ncols=True, leave=False, position=0, desc='Val') 

    phone_true_list = []
    phone_pred_list = []

    for i, data in enumerate(data_loader):

        mfccs, phonemes, len_mfccs, len_phonemes = data
        mfccs, phonemes = mfccs.to(device), phonemes.to(device) 

        with torch.inference_mode():
          out, lens = model(mfccs, len_mfccs)
    

        loss, lev_dist = criterion(out, phonemes, lens, len_phonemes, True)

        dist += lev_dist
        del mfccs, phonemes, out
        torch.cuda.empty_cache()
        
    dist /= len(data_loader)

    return loss, dist


In [27]:
def train_step(train_loader, model, optimizer, criterion, scheduler, scaler):
  
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    train_loss = 0

    for i, data in enumerate(train_loader):
        mfccs, phonemes, len_mfccs, len_phonemes = data
        mfccs, phonemes = mfccs.to(device), phonemes.to(device) 

        with torch.cuda.amp.autocast():
          out, lens = model(mfccs, len_mfccs)

        loss = criterion(out, phonemes, lens, len_phonemes)

        batch_bar.set_postfix(
            loss = f"{train_loss/ (i+1):.4f}",
            lr = f"{optimizer.param_groups[0]['lr']}"
        )

        train_loss += loss
        batch_bar.update()

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    
    batch_bar.close()
    train_loss /= len(train_loader)

    return train_loss

In [38]:
torch.cuda.empty_cache()
gc.collect()

best_val_dist = best_val_dist

name = "finetuning_second"

train_losses = []
validation_losses = []

for epoch in range(finetuning_config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, finetuning_config['epochs']))
    train_loss = train_step(train_loader, model, optimizer, criterion, scheduler, scaler)
    print("\tTrain Loss: {:.4f}".format(train_loss))
    
    validation_loss, levenshtein_distance = evaluate(val_loader, model, epoch, decoder)
    print("\tLevenshtein Distance: {}".format(levenshtein_distance))

    validation_losses.append(validation_loss)

    scheduler.step(levenshtein_distance)

    if levenshtein_distance < best_val_dist:
      path = "/content/drive/MyDrive/checkpoint_{}.pth".format(name) 
      print("Saving model")
      torch.save({'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'val_dist': levenshtein_distance, 
                  'epoch': epoch}, path)
      
      best_val_dist = levenshtein_distance
      wandb.save('checkpoint.pth')
    
    wandb.log({"train loss": train_loss, "levenshtein distance": levenshtein_distance, "validation loss": validation_loss})

run.finish()



Epoch 1/50


	Train Loss: -0.2970


	Levenshtein Distance: 15.016826923076923

Epoch 2/50


	Train Loss: -0.0894


	Levenshtein Distance: 16.22716346153846

Epoch 3/50


...

	Levenshtein Distance: 16.219951923076923

Epoch 6/50


	Train Loss: -0.0415


	Levenshtein Distance: 16.43329326923077
Epoch 00007: reducing learning rate of group 0 to 2.0000e-05.

Epoch 7/50


Train:  98%|█████████▊| 122/125 [00:58<00:01,  2.17it/s, loss=-0.0181, lr=2e-05]

KeyboardInterrupt: ignored

In [28]:
torch.cuda.empty_cache()
gc.collect()

best_val_dist = best_val_dist

name = "finetuning_second"

train_losses = []
validation_losses = []

for epoch in range(finetuning_config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, finetuning_config['epochs']))
    train_loss = train_step(train_loader, model, optimizer, criterion, scheduler, scaler)
    print("\tTrain Loss: {:.4f}".format(train_loss))
    
    validation_loss, levenshtein_distance = evaluate(val_loader, model, epoch, decoder)
    print("\tLevenshtein Distance: {}".format(levenshtein_distance))

    validation_losses.append(validation_loss)

    scheduler.step(levenshtein_distance)

    if levenshtein_distance < best_val_dist:
      path = "/content/drive/MyDrive/checkpoint_{}.pth".format(name) 
      print("Saving model")
      torch.save({'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'val_dist': levenshtein_distance, 
                  'epoch': epoch}, path)
      
      best_val_dist = levenshtein_distance
      wandb.save('checkpoint.pth')
    
    wandb.log({"train loss": train_loss, "levenshtein distance": levenshtein_distance, "validation loss": validation_loss})

run.finish()



Epoch 1/50


	Train Loss: -1.0754


	Levenshtein Distance: 7.778846153846154
Saving model

Epoch 2/50


	Train Loss: -0.8039


	Levenshtein Distance: 7.737980769230769
Saving model

Epoch 3/50


...

	Train Loss: -0.3563


	Levenshtein Distance: 7.201923076923077

Epoch 50/50


	Train Loss: -0.3714


	Levenshtein Distance: 7.1983173076923075
Saving model


levenshtein distance,██▇▇▆▅▅▅▅▅▄▄▄▄▄▃▄▃▃▂▃▂▂▂▃▃▃▃▂▂▂▁▂▁▁▁▁▁▁▁
train loss,▁▃▄▅▅▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇████▇
validation loss,▁▃▅▅▄▆▇▅▇▇▆▇▆▇▆▄▆▆▇▅█▆▆▇▇▇▇█▅▆▆▆▇▇▅▅▆▅▆▆
levenshtein distance,7.19832
train loss,-0.37139
validation loss,-0.24819


In [29]:
path = "/content/drive/MyDrive/checkpoint_first.pth"
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_dist': levenshtein_distance, 
            'epoch': epoch}, path)